In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import json
import cv2
import pickle
from utils import CocoImageFolder, to_var, show_images
from adaptive import Encoder2Decoder
from build_vocab import Vocabulary
from torch.autograd import Variable 
from torchvision import transforms

In [2]:
pretrained = 'models-attentive/adaptive-61.pkl'
images_path = './data/resized/test2014/'
vocab_path = './data/vocab.pkl'
caption_test_path = './data/annotations/image_info_test2014.json'
eval_size = 20
num_workers = 4

with open( vocab_path, 'rb') as f:
    vocab = pickle.load( f )

# Image transformation
transform = transforms.Compose([ 
    transforms.ToTensor(), 
    transforms.Normalize((0.485, 0.456, 0.406), 
                         (0.229, 0.224, 0.225))])

In [6]:
# Evaluation test image loader
cocoFolder = CocoImageFolder( images_path, caption_test_path, transform )
test_data_loader = torch.utils.data.DataLoader( 
        cocoFolder, 
        batch_size = 20, 
        shuffle = False, num_workers = 4,
        drop_last = False )
len(test_data_loader)

2039

In [7]:
# Define model and load pretrained
model = Encoder2Decoder( 256, len( vocab ), 512 )
model.load_state_dict( torch.load( pretrained ) )
model.eval()

if torch.cuda.is_available():
    model.cuda()

In [8]:
res = []

# every item in list is a batch of imgs, imgids, filenames
for i, (images, image_ids, filenames) in enumerate(test_data_loader):
    
#     if i > 0:
#         break
    
    images = to_var( images )
    generated_captions, attention, beta = model.sampler( images )
    
    captions = generated_captions.cpu().data.numpy()
    attention = attention.view( attention.size(0), attention.size(1), 7, 7 ).cpu().data.numpy()
    
    beta = beta.view( beta.size(0), beta.size(1) ).cpu().data.numpy()
    
    for image_idx in range( captions.shape[ 0 ] ):
        
        sampled_ids = captions[ image_idx ]
        sampled_caption = []
        
        for word_id in sampled_ids:
                
            word = vocab.idx2word[ word_id ]
            if word == '<end>':
                break
            else:
                sampled_caption.append( word )
        
        cap_len = len( sampled_caption )
        sentence = ' '.join( sampled_caption )
        image_attention = attention[ image_idx, :cap_len, :, : ]
        language_sentinel = beta[ image_idx, :cap_len ]
        
#         # Create the Graph
#         image_seq = []
#         titles = []
#         img = cv2.imread( images_path + filenames[ image_idx ] )
#         img = cv2.cvtColor( img, cv2.COLOR_BGR2RGB )
        
#         for cap_idx, word in enumerate( sampled_caption ):
            
#             height, width, _ = img.shape
            
#             heatmap = image_attention[ cap_idx ] * 7 # 7x7 attention map
#             heatmap = np.uint8( heatmap * 255 )
#             heatmap = cv2.applyColorMap( cv2.resize( heatmap, ( width, height ) ), cv2.COLORMAP_JET )
            
#             result = np.uint8( heatmap * 0.4  + img * 0.5  )
#             image_seq.append( result )
            
#             info = word + ' [%.2f]'%( language_sentinel[ cap_idx ] )
#             titles.append( info )
        
#         # Plot the attention images each row with 4 columns
#         print 'Attention Map of image:', filenames[ image_idx ]
        print 'Generated caption:', sentence
#         show_images( image_seq, 4, titles )
#         print ''
        res.append({"image_id": image_ids[image_idx], "caption": sentence})
        
# print(res)

with open('./results/test-61.json', 'w') as file:
    json.dump(res, file)

Generated caption: a man playing tennis on a tennis court .
Generated caption: a woman is playing tennis on a tennis court .
Generated caption: a woman holding a glass of wine and a glass of wine .
Generated caption: a desk with a laptop and a computer monitor
Generated caption: a living room with a couch , coffee table and a television .
Generated caption: a man in a white shirt and tie standing in front of a mirror .
Generated caption: a woman is feeding a giraffe in a zoo .
Generated caption: a plate of food with a sandwich and a fork .
Generated caption: two giraffes standing next to each other in a zoo .
Generated caption: a plate of food with a sandwich and a drink .
Generated caption: a zebra standing next to a tree in a field .
Generated caption: a young boy is standing in front of a fence .
Generated caption: a kitchen with a stove and a microwave
Generated caption: a couple of elephants standing next to each other .
Generated caption: a hot dog with a bun on a plate .
Generat

Generated caption: a man standing on a beach holding a kite .
Generated caption: a traffic light on a snowy street in the woods .
Generated caption: a living room with a couch , coffee table and a television .
Generated caption: a man wearing a suit and tie .
Generated caption: a street sign with a red light on it .
Generated caption: a laptop computer sitting on top of a desk .
Generated caption: a black and white motorcycle parked in a parking lot .
Generated caption: two giraffes standing next to a tree in a forest .
Generated caption: a man and a woman are skiing on a snowy slope .
Generated caption: a group of people are walking on the beach with surfboards .
Generated caption: a bathroom with a sink and a mirror
Generated caption: a plate of food with a fork and a fork .
Generated caption: a cat sitting on a suitcase on a sidewalk .
Generated caption: a group of people standing in a kitchen preparing food .
Generated caption: a bunch of suitcases are stacked on a table .
Generate

Generated caption: a small table with a chair and a coffee table
Generated caption: two women are sitting at a table with a plate of food .
Generated caption: a man standing in a field with a frisbee in the park .
Generated caption: a young boy playing soccer on a field .
Generated caption: a man riding a snowboard down a snow covered slope .
Generated caption: a man playing tennis on a tennis court .
Generated caption: a man in a red jacket is on a snow board
Generated caption: a large kitchen with a large wooden table with a large white plate .
Generated caption: a woman holding a cell phone up to her ear .
Generated caption: a herd of zebra standing in a field .
Generated caption: a man riding a bike down a street next to a bike .
Generated caption: a glass of wine and a bottle of wine on a table .
Generated caption: a man holding a frisbee on a beach .
Generated caption: a plate of food with a bowl of soup and a bowl of soup .
Generated caption: a train car with a train on it 's si

Generated caption: a woman holding a brown and white dog on a leash .
Generated caption: a large airplane sitting on top of a tarmac .
Generated caption: a laptop computer sitting on top of a wooden table .
Generated caption: a group of people playing a game on a nintendo wii .
Generated caption: a group of people playing a game of baseball .
Generated caption: a traffic light with a street sign on it .
Generated caption: a blue and white fire hydrant on a sidewalk .
Generated caption: a cat is laying on a bed with a cat .
Generated caption: a plate of food with broccoli and meat on a table .
Generated caption: a group of people sitting around a table with a laptop .
Generated caption: a traffic light on a city street at night .
Generated caption: a bus is traveling down a river .
Generated caption: a man holding a tennis racket on a tennis court .
Generated caption: a group of people riding bikes down a street .
Generated caption: a man in a red shirt is doing a trick on a skateboard 

Generated caption: a bus is parked in a parking lot .
Generated caption: a bus is driving down a street with people walking around .
Generated caption: a group of people sitting on a beach under an umbrella .
Generated caption: a person is playing a game on a television .
Generated caption: a man is holding a banana in a tree .
Generated caption: a park bench sitting in the middle of a park .
Generated caption: a woman walking down a street holding an umbrella .
Generated caption: a large elephant standing in a river next to a river .
Generated caption: a clock tower with a clock on top of it 's sides .
Generated caption: a living room with a fireplace and a coffee table
Generated caption: a brown bear sitting on a rock in a forest .
Generated caption: a clock tower with a clock on top of it 's sides .
Generated caption: a herd of zebra standing in a field .
Generated caption: a plate of food on a table with a fork .
Generated caption: a piece of cake on a plate with a fork .
Generated

...

Generated caption: a red suitcase and a red suitcase and a red suitcase .
Generated caption: a group of people walking down a street .
Generated caption: a room with a television , a table , and a television .
Generated caption: a woman sitting on a couch with a cat .
Generated caption: a person is looking at a dog in a kitchen .
Generated caption: a bed with a white blanket and a white bed
Generated caption: a street scene with a traffic light and a tree .
Generated caption: a group of people standing on a mountain side .
Generated caption: a man sitting on a couch with a remote control .
Generated caption: a woman is holding a blue umbrella and a man in a dress shirt and tie .
Generated caption: a pizza with a slice missing on it .
Generated caption: a group of people standing in a field flying kites .
Generated caption: a man sitting in a chair with a fire hydrant .
Generated caption: a woman holding a nintendo wii game controller .
Generated caption: a laptop computer sitting on to

Generated caption: a plate of food with a fork and a fork .
Generated caption: a couple of boxes that are sitting on a counter .
Generated caption: a glass of wine and a bottle of wine on a table .
Generated caption: a man sitting at a table with a plate of food .
Generated caption: a group of people flying kites on a beach .
Generated caption: a man in a red shirt is standing next to a fire hydrant .
Generated caption: a pizza with a slice missing on it .
Generated caption: a man and woman are sitting at a table with a wine glass .
Generated caption: a man flying a kite in a park .
Generated caption: a man in a suit and tie standing in a room .
Generated caption: a group of people are surfing in the water .
Generated caption: a black bear standing in a forest with a tree .
Generated caption: a cat is sitting on a bed with a cat .
Generated caption: a large elephant standing in a field of grass .
Generated caption: a sandwich with a sandwich on a plate with a fork .
Generated caption: 

Generated caption: a cat laying on a laptop computer .
Generated caption: a group of people standing in front of a building .
Generated caption: a close up of a person holding a toothbrush
Generated caption: a woman eating a slice of pizza on a plate .
Generated caption: a cat is laying on a bed with a cat .
Generated caption: a train is on a track near a building .
Generated caption: a group of birds are sitting on a table .
Generated caption: a car is parked in a parking lot .
Generated caption: a group of people standing next to a motorcycle .
Generated caption: a group of people standing on top of a snow covered slope .
Generated caption: a bench sitting in the woods next to a tree .
Generated caption: a tall building with a clock on it 's side .
Generated caption: a man and a woman are standing in front of a train .
Generated caption: a dog is laying on a couch with a dog .
Generated caption: a baseball player is holding a bat on a field .
Generated caption: a black bear standing 

Generated caption: a truck is parked in a field of grass .
Generated caption: a group of boats are floating in a river .
Generated caption: a bear and a bear in a zoo enclosure .
Generated caption: a blue and white bus parked next to a building .
Generated caption: a young boy is sitting on a skateboard .
Generated caption: a dog is looking at a mirror on a mirror .
Generated caption: a boat is floating in the water with a boat in the background .
Generated caption: a suitcase and a black suitcase on a table .
Generated caption: a bear is standing in the grass near a tree .
Generated caption: a man and a woman are playing a game on a wii .
Generated caption: a group of people standing on top of a snow covered slope .
Generated caption: a hot dog with a cup of coffee and a cup of coffee .
Generated caption: two women playing tennis on a tennis court .
Generated caption: a woman is playing tennis on a tennis court .
Generated caption: a bird sitting on a tree branch with a tree .
Generat

Generated caption: a clock is shown with a clock on it 's side .
Generated caption: a table with a plate of food and a bowl of soup .
Generated caption: a black and white photo of a umbrella on a sidewalk .
Generated caption: a man is skiing down a snowy slope .
Generated caption: a large jet flying in the air with a sky background
Generated caption: a group of people skiing down a snowy slope .
Generated caption: a laptop computer sitting on top of a desk .
Generated caption: a box of donuts and a box of donuts .
Generated caption: a kitchen with a stove and a microwave
Generated caption: a young boy holding a frisbee in a park .
Generated caption: a person is riding a bike on a city street .
Generated caption: a bowl of soup and a bowl of soup on a table .
Generated caption: a large elephant is standing in a field .
Generated caption: a man and a woman are skiing on a snowy slope .
Generated caption: a small elephant standing in a forest .
Generated caption: a plate of food with a fo

In [9]:
len(res)

40775